# Spaceship KNN Submission

Hello there whoever handles this aku lupa, ini cuma coba-coba aja si pake KNeighborsClassifier buat test submission. Feel free to modify ya, lov u makasi semangat. -Egar

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier


# Bring the Data In

In [ ]:
# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


# Model Training

In [ ]:
# Create pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalize data
    ('classifier', GradientBoostingClassifier(random_state=42))
])

# Define GridSearch parameters
parameters = {
    'classifier__n_estimators': [200],
    'classifier__learning_rate': [0.01],
    'classifier__max_depth': [10]
}

# Use StratifiedKFold to handle imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


In [ ]:
# GridSearchCV setup
grid_search = GridSearchCV(pipeline, parameters, cv=cv, scoring='roc_auc', verbose=1)
grid_search.fit(X_train, y_train)

# Evaluate the model
best_model = grid_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
print("Best parameters:", grid_search.best_params_)
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best parameters: {'classifier__learning_rate': 0.01, 'classifier__max_depth': 10, 'classifier__n_estimators': 200}
ROC-AUC on test set: 0.8970278874866537


In [ ]:
# Evaluation on the test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
grid_search.fit(X_df, y_df)

              precision    recall  f1-score   support

           0       0.89      0.96      0.92     26052
           1       0.77      0.54      0.63      6955

    accuracy                           0.87     33007
   macro avg       0.83      0.75      0.78     33007
weighted avg       0.86      0.87      0.86     33007

Confusion Matrix:
[[24964  1088]
 [ 3219  3736]]
Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        GradientBoostingClassifier(random_state=42))]),
             param_grid={'classifier__learning_rate': [0.01],
                         'classifier__max_depth': [10],
                         'classifier__n_estimators': [200]},
             scoring='roc_auc', verbose=1)

In [ ]:
# Predict probabilities for submission data
y_submission_proba = grid_search.predict_proba(X_submission)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

[0.04025906 0.68502159 0.04848224 ... 0.06029727 0.1995419  0.30512933]


,id,Exited
0,165034,0.040259
1,165035,0.685022
2,165036,0.048482
3,165037,0.249080
4,165038,0.315924
...,...,...
110018,275052,0.068760
110019,275053,0.111492
110020,275054,0.060297
110021,275055,0.199542


In [ ]:
submission_dict.to_csv('bankChurn_gbm.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>